# 라이브러리 로드 및 데이터 불러오기

In [10]:
import pandas as pd
import numpy as np
import warnings
from ast import literal_eval

warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 100)

In [5]:
movies = pd.read_csv('./input/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(3)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


# 데이터 가공하기

In [6]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [8]:
movies_df[['genres', 'keywords']][:2]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is..."


In [11]:
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [13]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


# 장르 컨텐츠 유사도 측정
  

만약 영화 A의 장르가 `[Action, Adventure, Fantasy, Science Fiction]`이고,  
B가 `[Adventure, Fantasy, Action]`으로 돼어있다면 어떻게 장르별 유사도를 측정할까요?  
  
여러가지 방법이 있을 수 있으나, 가장 간단한 방법은 genres를 문자열로 변경한 뒤,
이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것입니다.  
genres 컬럼을 기반으로 하는 컨텐츠 기반 필터링은 다음 단계로 구현하겠습니다.  
  
  1. 문자열로 변환된 genres 컬럼을 Count 기반으로 피처 벡터화 변환합니다.
  2. genres 문자열을 피처 벡터ㅗ하 행렬로 변환한 데이터 셋을 코사인 유사도를 통해 비교합니다. 이를 위해 데이터 셋의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성합니다.
  3. 장르 유사도가 높은 영화 중에서 평점이 높은 순으로 영화를 추천합니다.

장르 컬럼을 문자열로 변환한 뒤, 사이킷런의 CountVectorizer를 이용해 피처 벡터 행렬로 만들겠습니다.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.

movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


CountVectorizer로 변환해 4803개의 레코드와 276개의 개별 단어 피처로 구성도니 피처 벡터 행렬을 만듬.  
이렇게 생성된 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도를 계산하겠습니다.  

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_mat.shape)
print(genre_sim[:2])

(4803, 276)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


cosine_similarities() 호출로 생성된 genre_sim 객체는 movies_df 의 genre_literal 컬럼을 피처 벡터화한 행럴(genre_mat) 데이터의 행별로  
유사도 정보를 가지고 있으며, 결국은 movies_df DataFrame의 행별 장르 유사도 값을 가지고 있는 것입니다. movies_df를 장르 기준으로  
컨텐츠 기반 필터링ㄹ을 수행하려면 movies_df의 개별 레코드에 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 하는데,  
이를 위해 앞에서 생성한 genre_sim 객체를 이용합니다.